In [1]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Database connection parameters
db_params = {
    'host': '194.171.191.226',
    'port': '6379',
    'database': 'postgres',
    'user': 'group12',
    'password': 'blockd_2024group12_55'
}

# Function to establish a database connection
def get_db_connection(db_params):
    try:
        conn = psycopg2.connect(
            host=db_params['host'],
            port=db_params['port'],
            database=db_params['database'],
            user=db_params['user'],
            password=db_params['password']
        )
        return conn
    except psycopg2.DatabaseError as e:
        print(f"Error: {e}")
        return None

# Function to retrieve data from a specific table
def get_table_data(db_params, table_name):
    conn = get_db_connection(db_params)
    if conn is None:
        return None
    
    try:
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        conn.close()

In [3]:
# Function to retrieve data from a specific table
def get_table_data(db_params, table_name):
    conn = get_db_connection(db_params)
    if conn is None:
        return None
    
    try:
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        conn.close()

In [4]:
# Load the training, testing, and validation datasets from the database
train_table_name = 'group12_warehouse.train_table'
test_table_name = 'group12_warehouse.test_table'
validation_table_name = 'group12_warehouse.validation_table'

train_data = get_table_data(db_params, train_table_name)
test_data = get_table_data(db_params, test_table_name)
validation_data = get_table_data(db_params, validation_table_name)

if train_data is not None and test_data is not None and validation_data is not None:
    print(train_data.head())
    print(test_data.head())
    print(validation_data.head())
else:
    print("Failed to retrieve data from one or more tables.")

  Accident severity First Mode of Transport           Area Type  \
0               0.0      -0.350029334776985  0.3316993365651348   
1               0.0      -1.810821489630903  0.3316993365651348   
2               0.0      -0.350029334776985  0.3316993365651348   
3               0.0      -0.350029334776985  0.3316993365651348   
4               0.0      -0.350029334776985   -3.01477841455867   

      Light condition        Road Location       Road condition  \
0  0.6027962867863099  -0.8738398047923384    1.513226708878669   
1  0.6027962867863099   1.1443745117992685  -0.6608395121052414   
2  0.6027962867863099  -0.8738398047923384  -0.6608395121052414   
3  0.6027962867863099   1.1443745117992685  -0.6608395121052414   
4  -1.658935235535878   1.1443745117992685  -0.6608395121052414   

          Road surface        Road situation          Speed limit  \
0    1.784021822931772  -0.42384880344171616   1.9706356443111297   
1  -0.7354366365307541    1.0394771034977446  -0.1099889

/tmp/ipykernel_3021780/1785455050.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [5]:
# Split features and target variable
X_train = train_data.drop(columns=['Accident severity'])
y_train = train_data[['Accident severity']]
    
X_test = test_data.drop(columns=['Accident severity'])
y_test = test_data[['Accident severity']]
    
X_val = validation_data.drop(columns=['Accident severity'])
y_val = validation_data[['Accident severity']]

## Decision Tree Model

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Predict on the testing data
y_pred_test = clf.predict(X_test)

# Generate the classification report for the test set
report_test = classification_report(y_test, y_pred_test)

# Splitting the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # Assuming 60% train, 20% validation, 20% test

# Predict on the validation data
y_pred_val = clf.predict(X_val)

# Generate the classification report for the validation set
report_val = classification_report(y_val, y_pred_val)

print("Classification Report for Test Set:")
print(report_test)
print("\n")
print("Classification Report for Validation Set:")
print(report_val)


Classification Report for Test Set:
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       251
         1.0       0.78      0.98      0.87       251
         2.0       0.97      0.69      0.80       251

    accuracy                           0.89       753
   macro avg       0.90      0.89      0.88       753
weighted avg       0.90      0.89      0.88       753



Classification Report for Validation Set:
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       472
         1.0       0.94      0.99      0.96       429
         2.0       0.99      0.93      0.96       426

    accuracy                           0.98      1327
   macro avg       0.98      0.97      0.97      1327
weighted avg       0.98      0.98      0.98      1327

